In [7]:
import os
import pandas as pd
import sys
from pathlib import Path
import _whats_new_scripts as wn

# import global TDM functions
sys.path.insert(0, r'..\..\..\..\Resources\2-Python\global-functions')
import TDM

# Read Scenario Outputs 

In [8]:
tdmVersions = pd.DataFrame([
    [1, 12,'v910', 'WF TDM v9.1.0 - official', r'D:\GitHub\WF-TDM-v9x\Scenarios\WF TDM v910-Task2']
], columns = (['PROCESS','tdmVersionID','tdmVersion', 'tdmVersionWithDate', 'dirTdmVersionScenarios']))

segScenarios = pd.DataFrame([
    [ 1,'BY_2019'        ,'BY'            , 2019],
    [ 2,'OY_2023'        ,'OY'            , 2023],
    [13,'RTP_2032'       ,'RTP'           , 2032],
    [15,'RTP_2042'       ,'RTP'           , 2042],
    [16,'RTP_2050'       ,'RTP'           , 2050],
    [18,'TIP_2028'       ,'TIP'           , 2028]
], columns = (['scenarioID','scenarioName','scenarioGroup','scenarioYear']))

lstIncludeInAll = ['BY','OY','TIP']

# Merge Scenario Outputs

In [9]:
dirWork    = os.getcwd()

dfScenariosMaster = pd.DataFrame()
dfTransitShare    = pd.DataFrame()
dfRouteSummary    = pd.DataFrame()

scenarioID = 0

for index, rowT in tdmVersions.iterrows():

    if rowT['PROCESS']:

        for index, rowS in segScenarios.iterrows():

            # get scenario path
            _scenarioPath = os.path.join(rowT['dirTdmVersionScenarios'], rowS['scenarioName'])

            # check if scenario exists
            #print(_scenarioPath)
            if os.path.exists(_scenarioPath):
                
                try:
                    
                    _sID = rowS['scenarioID']

                    # round wasn't keeping float at 2 decimals, and was doing weird stuff
                    scenarioID = _sID

                    print(scenarioID)

                    # get values from Control Center to use to create filenames for outputs
                    _version = TDM.getControlCenterValue(os.path.join(_scenarioPath, "1ControlCenter.block"),'ModelVersion')
                    _scenario = TDM.getControlCenterValue(os.path.join(_scenarioPath, "1ControlCenter.block"),'ScenarioGroup')
                    _year = TDM.getControlCenterValue(os.path.join(_scenarioPath, "1ControlCenter.block"),'RunYear')
                    _runid = _version + '_' + _scenario + '_' + _year

                    try:
                        # get and join all transit route tables
                        dirRoute = _scenarioPath + '\\4_ModeChoice\\3_TransitAssign\\' + _runid + '_transit_brding_summary_route.csv'
                        if Path(dirRoute).is_file():
                            # file exist
                            dfRoute = pd.read_csv(dirRoute)
                            dfRoute['scenarioID'] = scenarioID
                            dfRoute.insert(0, 'scenarioID', dfRoute.pop('scenarioID')) # move scenario id to first column
                            dfRouteSummary = pd.concat([dfRouteSummary, dfRoute])
                        
                    except FileNotFoundError as f:
                        print(f)

                    try:
                        # get and join all transit share data
                        dfTransit = _scenarioPath + '\\4_ModeChoice\\4_Shares\\' + _runid + '_Shares_Summary_long.csv'
                        print(dfTransit)
                        if Path(dfTransit).is_file():
                            # file exists    
                            dfTransit = pd.read_csv(dfTransit)
                            dfTransit['scenarioID'] = scenarioID
                            dfTransit.insert(0, 'scenarioID', dfTransit.pop('scenarioID')) # move scenario id to first column
                            dfTransitShare = pd.concat([dfTransitShare, dfTransit])

                    except FileNotFoundError as f:
                        print(f)

                except FileNotFoundError as f:
                    print(f)
                    continue


1
D:\GitHub\WF-TDM-v9x\Scenarios\WF TDM v910-Task2\BY_2019\4_ModeChoice\4_Shares\WFv910_BY_2019_Shares_Summary_long.csv
2
D:\GitHub\WF-TDM-v9x\Scenarios\WF TDM v910-Task2\OY_2023\4_ModeChoice\4_Shares\WFv910_OY_2023_Shares_Summary_long.csv
13
D:\GitHub\WF-TDM-v9x\Scenarios\WF TDM v910-Task2\RTP_2032\4_ModeChoice\4_Shares\WFv910_RTP_2032_Shares_Summary_long.csv
15
D:\GitHub\WF-TDM-v9x\Scenarios\WF TDM v910-Task2\RTP_2042\4_ModeChoice\4_Shares\WFv910_RTP_2042_Shares_Summary_long.csv
16
D:\GitHub\WF-TDM-v9x\Scenarios\WF TDM v910-Task2\RTP_2050\4_ModeChoice\4_Shares\WFv910_RTP_2050_Shares_Summary_long.csv
18
D:\GitHub\WF-TDM-v9x\Scenarios\WF TDM v910-Task2\TIP_2028\4_ModeChoice\4_Shares\WFv910_TIP_2028_Shares_Summary_long.csv


# Create Intermediate Dataframes

In [10]:
# mode tags
mode_tags = {
    'Non-Motorized': ['Walk', 'Bike'],
    'Auto': ['Auto'],
    'Transit': ['CRT', 'LRT', 'BRT', 'LCL', 'COR', 'EXP']
}

# Function to generate multiple tags for each row
def assign_mode_groups(row):
    tags = []
    # Add specific tags based on mode tags dictionary
    for tag, modes in mode_tags.items():
        if row['MODE'] in modes:
            tags.append(tag)
    # Add the exact mode as its own tag
    tags.append(row['MODE'])
    # Ensure each tag appears only once
    return list(set(tags))

In [11]:
# new share
dfTransitShare[['tdmVersionWithDate']] = 'WF TDM v9.1.0 - official'
dfTransitShare = pd.merge(dfTransitShare,segScenarios, how='left',on='scenarioID')
dfTransitShare = dfTransitShare.drop(columns={'SUBAREAID'})

# old share
previousTransitShares = pd.read_csv(r"D:\GitHub\TDM-Documentation\v9x\v902\whats-new\data\intermediate\dfTransitSummaryPlotData.csv")
previousTransitShares = previousTransitShares[previousTransitShares['modeGroup']=='0: Total']
previousTransitShares = previousTransitShares.drop(columns={'modeGroup','tdmVersion'})

# combined share
combineTransitShare = pd.concat([dfTransitShare,previousTransitShares])

# assign mode tags
combineTransitShare['modeGroup'] = combineTransitShare.apply(assign_mode_groups, axis=1)
dfTransitShareAll = combineTransitShare.explode('modeGroup').reset_index(drop=True)

# create intermediate dataframe
dfTransitShareAll.to_csv("data/intermediate/dfTransitShareAll.csv")

# Test Plots

In [22]:
modelNew = 'WF TDM v9.1.0 - official'
modelOld = 'WF TDM v9.0.2 - official'

wn.update_plot(
    lstIncludeInAll,
    dfTransitShareAll,
    [modelNew, modelOld], 
    ['RTP'], 
    ['Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['pk','ok', 'Pk', 'Ok']
)

In [17]:
wn.update_plot_stackedarea(
    lstIncludeInAll,
    dfTransitShareAll,
    modelNew, 
    'RTP', 
    ['LCL','COR','EXP','BRT','LRT','CRT'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok', 'pk', 'ok']
)

In [18]:
mode_mapping = {4: 'LCL',5: 'EXP',6: 'COR',7: 'LRT',8: 'CRT',9: 'BRT'}

dfRouteSummary[['tdmVersionWithDate']] = 'WF TDM v9.1.0 - official'
dfRouteSummary = pd.merge(dfRouteSummary,segScenarios, how='left',on='scenarioID')
dfRouteSummary['ModeName'] = dfRouteSummary['Mode'].map(mode_mapping).fillna(dfRouteSummary['Mode'])


In [19]:
wn.update_plot_stackedarea_boardings(dfRouteSummary, modelNew, 'RTP', 'Boardings')

TypeError: update_plot_stackedarea_boardings() missing 1 required positional argument: 'BoardingsCol'